# Sea ice segmentation code using fastai v2

In [ ]:
%load_ext autoreload
%autoreload 2

Install Fastai: 

- Create a conda environment
- Install fastai using ```conda install -c fastchan fastai anaconda```
- For help with installation: https://docs.fast.ai/#Installing

In [ ]:
from fastai.vision.all import *
from metrics import *

In [ ]:
path = Path('../data/ice_image_data')

codes = ['Iceberg', 'Floe berg', 'Floe bit', 'Open water', 'Melt pond',
       'Level ice', 'Deformed ice', 'Ice floe', 'Pancake ice',
       'Underwater ice', 'Broken ice', 'Brash ice', 'Shore', 'Sky']


In [ ]:
def MeanIoU(y_pred:Tensor, y_true:Tensor, **kwargs): return IOU(y_pred, y_true, classes=len(codes), **kwargs)

MeanIoU.__repr__ = "MeanIoU"

# Dataset

In [ ]:
def label_func(fn): 
    return fn.parent.parent/f"{fn.parent.stem.replace('images', 'masks')}/{fn.stem}_mask.png"

def get_data_df(imgs_path):
    fnames = get_image_files(imgs_path)
    
    data = pd.DataFrame(dict(img_file=fnames))
    data.loc[:, 'mask_file'] = data['img_file'].apply(label_func)
    
    np.random.seed(1)
    valid_idxs = np.random.choice(np.arange(1, len(data)), int(0.2*len(data)))
    data.loc[:, 'is_valid'] = False
    data.loc[valid_idxs, 'is_valid'] = True
    return data 

In [ ]:
df = get_data_df(path/"journal_resized/images_journal_resized")
df

In [ ]:
df['is_valid'].value_counts()

In [ ]:
db = DataBlock(blocks = (ImageBlock, MaskBlock(codes)),
               get_x = ColReader('img_file'),
               get_y = ColReader('mask_file'),
               splitter = ColSplitter(col='is_valid'),
               item_tfms = [Resize((512, 512))],
               batch_tfms = [*aug_transforms(), Normalize.from_stats(*imagenet_stats)])
 
dls = db.dataloaders(df, bs=2, tfm_y=True)

# Train

In [ ]:
learn = unet_learner(dls, resnet34, metrics=[MeanIoU])
learn.fine_tune(6, base_lr=1e-2)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(15, lr_max=1e-2)